In [ ]:
!pip install -r requirements.txt -q

In [ ]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL
from sklearn.model_selection import KFold
from torch.utils.data import  DataLoader, TensorDataset, Dataset
from torch import nn
import timm
from tqdm.notebook import tqdm
import torch.optim as optim

## Exemplo de classificador com leitura do CSV. 

Etapas:

- Leitura CSV.
- Tratamento de imagens com lib PIL. 
- Leitura imagem. 
- Aplicacao Filtro para remocao background


Input do modelo : (Imagem, sexo, idade, etnia)

In [2]:
import pandas as pd
from PIL import Image, ImageChops
import numpy as np
import torch
from lib.ImageFIlter import treat_image_PIL
from sklearn.model_selection import KFold
from torch.utils.data import  DataLoader, TensorDataset, Dataset
from torch import nn
import timm
from tqdm.notebook import tqdm
import torch.optim as optim

In [ ]:
## Geracao dataset com aleatorio
ds_file = pd.read_csv('dataset/exemplo_csv_2.csv')
arr = np.random.choice([0, 1], size=216)

ds_file = ds_file[['path','image_id','ID','IDADE','SEXO','ETNIA']]
ds_file['oca'] = arr
ds_file.to_csv('dataset/oca.csv')

In [7]:
ds_file = pd.read_csv('dataset/oca.csv')


## Geracao Tensor com imagens filtradas

In [8]:
## Loop
img_dataset = np.ones((ds_file.shape[0],3,256,256),dtype=np.uint8)

j=0
for i in ds_file['path']:
    img_dataset[j]=treat_image_PIL('dataset/path/'+i,2)
    j+=1
tensor_imagem = torch.tensor(img_dataset)



## Geracao Tensor Age, sex, Etinia

Encoding: 

- Sexo: M=0 F=1

- Idade: Normalizar? Sim, dividir por 100 igual o artigo do Marco e Felipe fizeram. 

- Etnia: Encoding simples de categorias. Utilizaremos o Label Encoding para isso 

In [9]:
ds_id_sex_et= ds_file[['IDADE', 'SEXO', 'ETNIA']]
ds_id_sex_et['SEXO'] = ds_file['SEXO'].map({ 'M':0, 'F':1})
ds_id_sex_et['IDADE'] = ds_file['IDADE']/100

/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_49789/1074131203.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['SEXO'] = ds_file['SEXO'].map({ 'M':0, 'F':1})
/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_49789/1074131203.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['IDADE'] = ds_file['IDADE']/100


In [10]:
## Encoding variavel ETNIA -> Usando label Encoding
from sklearn.preprocessing import LabelEncoder

# Create a sample dataframe with categorical data

#print(f"Before Encoding the Data:\n\n{ds_id_sex_et['ETNIA']}\n")

# Create a LabelEncoder object
le = LabelEncoder()

# Fit and transform the categorical data
ds_id_sex_et['ETNIA'] = le.fit_transform(ds_id_sex_et['ETNIA'])
ds_id_sex_et

/var/folders/30/pgp77_r92w5_rn1ghx4kn51h0000gn/T/ipykernel_49789/2772605160.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ds_id_sex_et['ETNIA'] = le.fit_transform(ds_id_sex_et['ETNIA'])


,IDADE,SEXO,ETNIA
0,0.85,0,1
1,0.32,1,1
2,0.60,1,1
3,0.57,1,4
4,0.57,1,1
...,...,...,...
211,0.35,1,3
212,0.72,0,2
213,0.71,1,2
214,0.49,0,2


## Geracao de rótulos para classificação

Colunas utilizadas para classificacao (Labels)

labels = [ 'FA', 'TA/Flutter',
       'TPSV/TS', 'TV/FV', 'BAV 2º/3º/Avanc./BS', 'Supra ST',
       'Corrente de Lesao', 'Extrassistole', 'BRD', 'BRE', 'MP', 'Normal',
       'Outros', 'Exclusão']

In [ ]:
##
ds_file.columns
labels = [ 'oca']
ds_labels = ds_file[labels]
ds_labels=ds_labels.astype(int)#,'False':0})
tensor_label = torch.tensor(np.array(ds_labels))
tensor_label

### Resumo:

model(tensor_imagem,ds_id_sex_et)

labels : tensor_label

## Criacao Subset 



In [13]:
class Subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
    """
    def __init__(self, dataset, indices):
        self.dataset = dataset
        self.indices = indices

    def __getitem__(self, idx):
        return self.dataset[self.indices[idx]]

    def __len__(self):
        return len(self.indices)

    @property
    def classes(self):
        return self.data.classes

## K-fold = 10


In [14]:
kf = KFold(n_splits=5)
kf.get_n_splits(tensor_imagem)
print(kf)
#for i in enumerate(kf.split(tensor_imagem)):
#    print(i)

KFold(n_splits=5, random_state=None, shuffle=False)


## Criacao do dataset de treino e validacao com batchsize

In [15]:
BATCH_SIZE = 16
train_dataset = TensorDataset(tensor_imagem, tensor_label)
train_loader_img = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [16]:
train_dataset[0][0]

tensor([[[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)

## Create a model to concat layers

O modelo utilizado será o mesmo do artigo: Artificial Intelligence-Driven Screening System for Rapid Image-Based Classification of 12-Lead ECG Exams: A Promising Solution for Emergency Room Prioritization 

Desenvolvido conforme a figura abaixo: 

![image.png](attachment:image.png)

In [ ]:
class ECGClassifierAIntelligence(nn.Module):    
    def __init__(self, num_classes=53):
        super(ECGClassifierAIntelligence, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        enet_out_size = int(32*(250*250)/25)      # 32 canais de imagens 250x250 passando por um maxpooling de 5x5 (32x250x250)/(5x5)

        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=32)
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=1,out_channels=16,kernel_size=(3,3)), ## 3, 256,256 -> 32, 254,254
        nn.ReLU(),
        nn.Conv2d(in_channels=16,out_channels=16,kernel_size=(3,3)), ## 32, 254,254 -> 64, 252,252 
        nn.ReLU(), nn.MaxPool2d((2,3)))
        
        self.layer1 = nn.Sequential(nn.Conv2d(in_channels=3,out_channels=16,kernel_size=(3,3)), ## 3, 256,256 -> 32, 254,254
        nn.ReLU(),
        nn.Conv2d(in_channels=16,out_channels=16,kernel_size=(3,3)), ## 32, 254,254 -> 64, 252,252 
        nn.ReLU(), nn.MaxPool2d((3,3)))



        self.layer2= 
        self.model = nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=32,kernel_size=(3,3)), ## 3, 256,256 -> 32, 254,254
        nn.ReLU(),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=(3,3)), ## 32, 254,254 -> 64, 252,252 
        nn.ReLU(),
        nn.Conv2d(in_channels=64,out_channels=32,kernel_size=(3,3)), ## 32, 250,250 ,
        nn.MaxPool2d(5),
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes), nn.Sigmoid())
    
         # saida como linear 
    
    def forward(self, img_tensor, sex,etnia,age):
        # Connect these parts and return the output
        #x = self.features(x)
        output = self.model(img_tensor) ## saída linear 
        concatenated = torch.cat(output,sex,etnia,age)
        return output

SyntaxError: invalid syntax (848933890.py, line 21)

In [ ]:
class ECGClassifierResnet(nn.Module):    
    def __init__(self, num_classes=53):
        super(ECGClassifierResnet, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        enet_out_size = int(32*(250*250)/25)      # 32 canais de imagens 250x250 passando por um maxpooling de 5x5 (32x250x250)/(5x5)

        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=32)
        self.model = nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=32,kernel_size=(3,3)), ## 3, 256,256 -> 32, 254,254
        nn.ReLU(),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=(3,3)), ## 32, 254,254 -> 64, 252,252 
        nn.ReLU(),
        nn.Conv2d(in_channels=64,out_channels=32,kernel_size=(3,3)), ## 32, 250,250 ,
        nn.MaxPool2d(5),
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes), nn.Sigmoid())
    
         # saida como linear 
    
    def forward(self, img_tensor, sex,etnia,age):
        # Connect these parts and return the output
        #x = self.features(x)
        output = self.model(img_tensor) ## saída linear 
        concatenated = torch.cat(output,sex,etnia,age)
        return output

In [22]:
#from models import ECGClassifierResnet

class ECGClassifierResnet(nn.Module):
    def __init__(self, num_classes=5):
        super(ECGClassifierResnet, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True) 
        self.base_model=timm.create_model('resnet50d.ra4_e3600_r224_in1k',pretrained=True)
        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=5,pretrained=True)


        self.features = nn.Sequential(*list(self.base_model.children())[:-1])

        enet_out_size = 2048        # Make a classifier
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size,1)
        ) # saida como Sigmoid multilabel

    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

In [ ]:
class ECGClassifierResnet(nn.Module):    
    def __init__(self, num_classes=53):
        super(ECGClassifierResnet, self).__init__()
        # Where we define all the parts of the model
        #self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        enet_out_size = int(32*(250*250)/25)      # Make a classifier

        #self.base_model = timm.create_model('vit_mediumd_patch16_reg4_gap_256.sbb2_e200_in12k_ft_in1k',num_classes=32)
        self.model = nn.Sequential(
        nn.Conv2d(in_channels=3,out_channels=32,kernel_size=(3,3)), ## 3, 256,256 -> 32, 254,254
        nn.ReLU(),
        nn.Conv2d(in_channels=32,out_channels=64,kernel_size=(3,3)), ## 32, 254,254 -> 64, 252,252 
        nn.ReLU(),
        nn.Conv2d(in_channels=64,out_channels=32,kernel_size=(3,3)), ## 32, 250,250 ,
        nn.MaxPool2d(5),
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes), nn.Sigmoid())
    
         # saida como linear 
    
    def forward(self, x):
        # Connect these parts and return the output
        #x = self.features(x)
        output = self.model(x)
        return output

## Train - Valid Loop



In [23]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Device: " + str(device))
def simple_loop(model, train_image, val_image, epochs):
    # Simple training loop
    num_epochs = epochs
    train_losses, val_losses = [], []
    lim_loss = 1.5

    #model = modelo( num_classes=5)
    model.to(device)
    criterion =  nn.BCEWithLogitsLoss()

    # Optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01)


    for epoch in range(num_epochs):
        # Training phase
        model.train()
        running_loss_train = 0.0
        for images, labels in tqdm(train_image, desc='Training loop'):
            # Move inputs and labels to the device
            images = images.to(torch.float)
            image, label = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(image)
            loss_train = criterion(outputs.float(), label.float())
            loss_train.backward()
            optimizer.step()
            running_loss_train += loss_train.item() * label.size(0)
        train_loss = running_loss_train / len(train_image.dataset)
        train_losses.append(train_loss)
        model.eval()
        running_loss_valid = 0.0
        rotulos =[] 
        output_model =[]
        with torch.no_grad():
            for images, labels in tqdm(val_image, desc='Validation loop'):
                # Move inputs and labels to the device
                images = images.to(torch.float)
                images, label = images.to(device), labels.to(device)
                rotulos.append(label.cpu().data.numpy())
                outputs = model(images)
                loss_valid = criterion(outputs.float(), label.float())
                output_model.append(outputs.cpu().data.numpy())
                running_loss_valid += loss_valid.item() * label.size(0)
        val_loss = running_loss_valid / len(val_image.dataset)
        val_losses.append(val_loss)
        #val_acc = accuracy_score(rotulos,output_model)
        print(f'Val accuracy {1}')
        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")



    return train_losses, val_losses, model,outputs, labels

Device: cpu


In [24]:
train_dataset[0]

(tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8),
 tensor([1]))

In [25]:

train_loss_total = []
val_loss_total =[]
all_models =[]
epochs = 30
## create first model.
for i, (train_index, test_index) in enumerate(kf.split(train_dataset)):
    print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")
    ## init train test for folder
    print(train_index)
    train_dataset_part = Subset( train_dataset, train_index)
    val_dataset_part = Subset( train_dataset, test_index)

    print(train_dataset_part)
    train_loader_img = DataLoader(train_dataset_part, batch_size=5, shuffle=True)
    val_loader_img = DataLoader(val_dataset_part, batch_size=5, shuffle=True)

    model= ECGClassifierResnet( num_classes=1)
    all_models.append(model)
    print(f'Train and valid for Fold {i}')
    t, l,_,outputs,labels = simple_loop(model, train_loader_img,val_loader_img, epochs)
    ## Evaluate model.
   
    train_loss_total.append(t)
    val_loss_total.append(l)

Fold 0:
[ 44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61
  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79
  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97
  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151
 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169
 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205
 206 207 208 209 210 211 212 213 214 215]
Train and valid for Fold 0


Training loop:   0%|          | 0/35 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
train_loss_total

[]